# Explorative Datenanalyse der Kriminalität in Chicago
----

# Konfiguration

In [1]:
%matplotlib inline
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

In [9]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

<style>body {
    margin: 0;
    font-family: Helvetica;
}
table.dataframe {
    border-collapse: collapse;
    border: none;
}
table.dataframe tr {
    border: none;
}
table.dataframe td, table.dataframe th {
    margin: 0;
    border: 1px solid white;
    padding-left: 0.25em;
    padding-right: 0.25em;
}
table.dataframe th:not(:empty) {
    background-color: #fec;
    text-align: left;
    font-weight: normal;
}
table.dataframe tr:nth-child(2) th:empty {
    border-left: none;
    border-right: 1px dashed #888;
}
table.dataframe td {
    border: 2px solid #ccf;
    background-color: #f4f4ff;
}

table.dataframe td,
table.dataframe th {
    color: black;
}h3 {
    color: white;
    background-color: black;
    padding: 0.5em;
}
</style>

In [4]:
%%time
crimes = pd.read_csv(f"data/crimes.csv")

CPU times: user 33.5 s, sys: 5.11 s, total: 38.6 s
Wall time: 38.7 s


In [5]:
list(crimes)

['ID',
 'Case Number',
 'Date',
 'Block',
 'IUCR',
 'Primary Type',
 'Description',
 'Location Description',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'Community Area',
 'FBI Code',
 'X Coordinate',
 'Y Coordinate',
 'Year',
 'Updated On',
 'Latitude',
 'Longitude',
 'Location']

In [8]:
crimes.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10000092,HY189866,03/18/2015 07:44:00 PM,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,False,False,1111,11.0,28.0,25.0,04B,1144606.0,1903566.0,2015,02/10/2018 03:50:01 PM,41.891399,-87.744385,"(41.891398861, -87.744384567)"
1,10000094,HY190059,03/18/2015 11:00:00 PM,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,True,False,725,7.0,15.0,67.0,26,1166468.0,1860715.0,2015,02/10/2018 03:50:01 PM,41.773372,-87.665319,"(41.773371528, -87.665319468)"
2,10000095,HY190052,03/18/2015 10:45:00 PM,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,222,2.0,4.0,39.0,08B,1185075.0,1875622.0,2015,02/10/2018 03:50:01 PM,41.813861,-87.596643,"(41.81386068, -87.596642837)"
3,10000096,HY190054,03/18/2015 10:30:00 PM,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,False,False,225,2.0,3.0,40.0,08B,1178033.0,1870804.0,2015,02/10/2018 03:50:01 PM,41.800802,-87.622619,"(41.800802415, -87.622619343)"
4,10000097,HY189976,03/18/2015 09:00:00 PM,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,False,1113,11.0,28.0,25.0,03,1144920.0,1898709.0,2015,02/10/2018 03:50:01 PM,41.878065,-87.743354,"(41.878064761, -87.743354013)"


# Straftatentypen und deren Häufigkeit

In [204]:
crimes.groupby(crimes['Primary Type']).size().sort_values(ascending=False)

Primary Type
THEFT                                1417696
BATTERY                              1232015
CRIMINAL DAMAGE                       771356
NARCOTICS                             710760
OTHER OFFENSE                         418441
ASSAULT                               418375
BURGLARY                              387936
MOTOR VEHICLE THEFT                   314156
DECEPTIVE PRACTICE                    262300
ROBBERY                               255578
CRIMINAL TRESPASS                     193342
WEAPONS VIOLATION                      70639
PROSTITUTION                           68322
PUBLIC PEACE VIOLATION                 47775
OFFENSE INVOLVING CHILDREN             45301
CRIM SEXUAL ASSAULT                    27075
SEX OFFENSE                            25081
INTERFERENCE WITH PUBLIC OFFICER       15136
GAMBLING                               14422
LIQUOR LAW VIOLATION                   14061
ARSON                                  11151
HOMICIDE                                94

# Straftaten vs. Verhaftungen

In [11]:
c = crimes
c = c.groupby(['Primary Type', 'Arrest']).size().sort_values(ascending=False)
c = c.unstack()
c['Total'] = c[False] + c[True]
c = c.sort_values(ascending=False, by='Total')
c = (c[True] / (c[True] + c[False]))
c = c.sort_values(ascending=False)
c = c.astype(float).map("{:.2%}".format)

### Häufige Verhaftung
Drogendelikte enden überproportional oft in Verhaftungen. 

In [12]:
c.head(10)

Primary Type
PROSTITUTION                         99.60%
NARCOTICS                            99.41%
PUBLIC INDECENCY                     99.38%
GAMBLING                             99.28%
LIQUOR LAW VIOLATION                 99.10%
CONCEALED CARRY LICENSE VIOLATION    93.29%
INTERFERENCE WITH PUBLIC OFFICER     91.72%
OBSCENITY                            82.76%
WEAPONS VIOLATION                    79.58%
CRIMINAL TRESPASS                    73.25%
dtype: object

### Seltene Verhaftung

In [13]:
c.tail(10)

Primary Type
ARSON                  13.02%
HUMAN TRAFFICKING      12.24%
THEFT                  11.95%
KIDNAPPING             11.12%
ROBBERY                 9.70%
MOTOR VEHICLE THEFT     9.13%
CRIMINAL DAMAGE         7.06%
NON-CRIMINAL            6.13%
BURGLARY                5.73%
DOMESTIC VIOLENCE        nan%
dtype: object